## import packages

In [31]:
import os
from datetime import datetime, timedelta
import pandas as pd

from pandas import DataFrame, Series
from typing import List

from dotenv import load_dotenv

load_dotenv()

True

## Variables

### Env

In [32]:
INPUT_FOLDER: str = os.getenv('INPUT_FOLDER_PATH')

### other

In [33]:
time_columns: list = ['System', 'Queue', 'Ring', 'Talk', 'Hold', 'ACW', 'Consult', 'Disposition', 'Duration']
vdn_flow_columns: list = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
call_work_codes_columns: list = ['1.1', '2.1', '3.1', '4.1', '5.1']

## Functions

### summarize_unused_columns

In [34]:
def summarize_unused_columns(dataset_column_index: pd.core.indexes.base.Index) -> List[str]:
   """
   
   """
   unused_column_names: List[str] = [col_name for col_name in dataset_column_index if 'unnamed' in str(col_name).lower()]
   return unused_column_names

### filter_unused_columns

In [35]:
def filter_unused_columns(dataset: DataFrame) -> DataFrame:
    """
    
    """
    input_dataset: DataFrame = dataset.copy()
    unused_column_names: List[str] = summarize_unused_columns(dataset.columns)
    clean_df: DataFrame = input_dataset.drop(labels=unused_column_names, axis='columns')
    return clean_df

### set_datetime_dtype_values

In [36]:
def set_datetime_dtype_values(dataset: DataFrame, columns: List[str]) -> DataFrame:
    """
    
    """
    for col_name in columns:
        dataset[col_name] = dataset[col_name].map(lambda str_value: datetime.strptime(str_value, "%Y-%m-%d %H:%M:%S"))

    return dataset

### timedelta_from_string

In [37]:
def timedelta_from_string(timestring: str) -> timedelta:
    """
    
    """
    time_obj = datetime.strptime(timestring,"%H:%M:%S")
    timedelta_obj: timedelta = timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second)
    return timedelta_obj

### timestring_to_seconds

In [38]:
def timestring_to_seconds(dataset: DataFrame, columns: List[str]) -> DataFrame:
    """
    
    """
    for col_name in columns:
        dataset[col_name] = dataset[col_name].map(lambda timestring: timedelta_from_string(timestring))

    return dataset

### get_avg_time

In [39]:
def get_avg_time(dataset: DataFrame, columns: List[str]) -> timedelta:
    """
    
    """
    subset: DataFrame = dataset[columns].copy()
    subset['row_sum'] = subset.sum(axis='columns')
    average: timedelta = subset['row_sum'].mean()
    return average

## core logic

### test read

In [40]:
test_df: DataFrame = pd.read_excel(os.path.join(INPUT_FOLDER, 'Week 2 -Mei.xlsx'), header=1)
test_df.head()

,Unnamed: 0,CallId,Segment,Start,Stop,Calling Party,Trunk Group,Dialed Number,Answered Agent,Original Agent,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,1.1,2.1,3.1,4.1,5.1
0,NaN,597172,1,2025-05-12 08:00:10,2025-05-12 08:10:33,0634233477,101 (Vodafone01),8008 (SGD Serviceloket),95143 (Erdem Eser),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,597158,1,2025-05-12 08:01:47,2025-05-12 08:04:47,7552 (Pieter-Jan Kersbergen),NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,597157,1,2025-05-12 08:02:25,2025-05-12 08:04:46,0180769288,101 (Vodafone01),8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,597167,1,2025-05-12 08:04:29,2025-05-12 08:08:30,"5818 (Zodenkamp, B (Brigitte))",NaN,8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,597175,1,2025-05-12 08:04:33,2025-05-12 08:12:56,7356 (Sjouke Hettema),NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### test filter unused columns

In [41]:
test_df = filter_unused_columns(dataset=test_df)
test_df.columns

Index([          'CallId',          'Segment',            'Start',
                   'Stop',    'Calling Party',      'Trunk Group',
          'Dialed Number',   'Answered Agent',   'Original Agent',
       'Call Disposition',           'Vector',         'Priority',
                  'Split',            'Level',           'System',
                  'Queue',             'Ring',             'Talk',
                   'Hold',              'ACW',          'Consult',
            'Disposition',         'Duration',                  1,
                        2,                  3,                  4,
                        5,                  6,                  7,
                        8,                  9,              '1.1',
                    '2.1',              '3.1',              '4.1',
                    '5.1'],
      dtype='object')

### test set datetime dtype

In [42]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-12 08:00:10 <class 'str'>


In [43]:
test_df = set_datetime_dtype_values(test_df, columns=['Start', 'Stop'])

In [44]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-12 08:00:10 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


### test timestring to seconds

In [45]:
test_value = test_df['System'][0]
print(test_value, type(test_value))

00:00:40 <class 'str'>


In [46]:
result = timedelta_from_string(test_value)
result

datetime.timedelta(seconds=40)

In [47]:
test_df = timestring_to_seconds(test_df, time_columns)

In [48]:
test_value = test_df['System'][0]
print(test_value, type(test_value))

0 days 00:00:40 <class 'pandas._libs.tslibs.timedeltas.Timedelta'>


### Test aggregations

In [52]:
test_df.head()

,CallId,Segment,Start,Stop,Calling Party,Trunk Group,Dialed Number,Answered Agent,Original Agent,Call Disposition,...,6,7,8,9,1.1,2.1,3.1,4.1,5.1,Datum
0,597172,1,2025-05-12 08:00:10,2025-05-12 08:10:33,0634233477,101 (Vodafone01),8008 (SGD Serviceloket),95143 (Erdem Eser),NaN,Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
1,597158,1,2025-05-12 08:01:47,2025-05-12 08:04:47,7552 (Pieter-Jan Kersbergen),NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
2,597157,1,2025-05-12 08:02:25,2025-05-12 08:04:46,0180769288,101 (Vodafone01),8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
3,597167,1,2025-05-12 08:04:29,2025-05-12 08:08:30,"5818 (Zodenkamp, B (Brigitte))",NaN,8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
4,597175,1,2025-05-12 08:04:33,2025-05-12 08:12:56,7356 (Sjouke Hettema),NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12


In [51]:
test_df["Datum"] = pd.to_datetime(test_df['Start']).dt.date

In [54]:
subset = test_df['Datum'].unique()
for date in subset:
    print(date.day)

12
13
14
15
16


In [64]:
tt = test_df[(test_df['Segment'] != 1) & (test_df['Dialed Number'] != '8008 (SGD Serviceloket)')]
tt

,CallId,Segment,Start,Stop,Calling Party,Trunk Group,Dialed Number,Answered Agent,Original Agent,Call Disposition,...,6,7,8,9,1.1,2.1,3.1,4.1,5.1,Datum
22,597286,2,2025-05-12 08:32:47,2025-05-12 08:40:02,95021 (Rosie Bhagwanie),NaN,8031 (SGDIVRD),95143 (Erdem Eser),95021 (Rosie Bhagwanie),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
104,598431,2,2025-05-12 10:30:15,2025-05-12 10:50:36,95163 (Sakoen Dwarka),NaN,8031 (SGDIVRD),95121 (Jordy Cruz),95163 (Sakoen Dwarka),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
113,598474,2,2025-05-12 10:45:06,2025-05-12 10:55:32,95163 (Sakoen Dwarka),NaN,8031 (SGDIVRD),95143 (Erdem Eser),95163 (Sakoen Dwarka),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
136,598923,2,2025-05-12 11:36:31,2025-05-12 11:45:03,95161 (Mariel Okkerse),NaN,8031 (SGDIVRD),95022 (Yamani Mondt),95161 (Mariel Okkerse),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
142,598981,2,2025-05-12 11:50:02,2025-05-12 11:51:37,95020 (Patricia Ringers),NaN,8031 (SGDIVRD),95123 (Matthew Hall),95020 (Patricia Ringers),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
148,599089,2,2025-05-12 11:56:20,2025-05-12 12:03:30,95020 (Patricia Ringers),NaN,8031 (SGDIVRD),95022 (Yamani Mondt),95020 (Patricia Ringers),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
154,599132,2,2025-05-12 12:02:45,2025-05-12 12:08:00,95174 (Natalie Ducoffre),NaN,8031 (SGDIVRD),NaN,95174 (Natalie Ducoffre),Abandoned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
211,600716,2,2025-05-12 15:42:04,2025-05-12 15:43:34,95392 (DGJ Bedienpost 2),NaN,8000 (SGD Hoofdnummer),95174 (Natalie Ducoffre),95392 (DGJ Bedienpost 2),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
212,600716,3,2025-05-12 15:43:23,2025-05-12 15:43:34,95174 (Natalie Ducoffre),NaN,8031 (SGDIVRD),95143 (Erdem Eser),95174 (Natalie Ducoffre),Abandoned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
222,600983,2,2025-05-12 16:21:31,2025-05-12 16:27:29,95174 (Natalie Ducoffre),NaN,8031 (SGDIVRD),95122 (Ferdi Guler),95174 (Natalie Ducoffre),Answered,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-12
